# Lab 6

In [ ]:
# %pip install geemap

In [ ]:
# Import packages
import ee
import geemap

from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
geemap.ee_initialize()

In [ ]:
# Check initialize
Map = geemap.Map()
#Map

# Question 1

Create a map to visualize NOAA GFS Temperature Data and add a color bar and NOAA logo to the map.


In [ ]:
Map1 = geemap.Map()

# Load Feature Collection
dataset = ee.ImageCollection('NOAA/GFS0P25').filter(ee.Filter.date('2024-03-01', '2024-03-02'))

temperature = dataset.select('temperature_2m_above_ground');

# Config. visualization
vis_params = {
    'min': -40.0,
    'max': 35.0,
    'palette':  ['blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
    }

# set style border
style = {'color': '0000ffff', 'width': 2, 'lineType': 'solid',
         'fillColor': '0000ff00'}

# Add text to the map
text = "Created by Pablo Miranda"
Map1.add_text(text, fontsize=20, position='bottomright')

# Add a logo to the map
html = """
<img src="https://i.imgur.com/spILFEi.png">
"""
Map1.add_html(html, position="bottomright")

# Add Layers
Map1.addLayer(temperature, vis_params,  'Temperature data')
Map1

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

![](https://i.imgur.com/3KIRhfU.jpeg)

# Question 2

Linked Maps: Create a 2*2 linked map to visualize the Landsat imagery (ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')) with different band combinations.

In [ ]:
Map2 = geemap.Map()

# Load the bounds of Ordesa National Park in Spain
ordesa = ee.FeatureCollection("WCMC/WDPA/current/polygons").filter(ee.Filter.eq('NAME', 'Ordesa y Monte Perdido'))

# Get centroid
centroid = ordesa.geometry().centroid().coordinates()
centroid

In [ ]:
Map2 = geemap.Map()

# Load dataset
image = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')

# Set vis parms
vis_params = [
    {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 120, 'gamma': 1.3},
    {'bands': ['B3', 'B2', 'B1'], 'min': 0, 'max': 120, 'gamma': 1.3},
    {'bands': ['B5', 'B4', 'B3'], 'min': 0, 'max': 120, 'gamma': 1.3},
    {'bands': ['B7', 'B5', 'B3'], 'min': 0, 'max': 120, 'gamma': 1.3},
]

# Add labels
labels = [
    'False Color (B4/B3/B2)',
    'Natural Color (B3/B2/B1)',
    'False Color (Vegetative Abalysis) (B5/B4/B3)',
    'False Color (Urban) (B7/B5/B3)',
]
Map2.addLayer(ordesa, {}, 'Ordesa')

# Add linked map 2*2
geemap.linked_maps(
    rows=2,
    cols=2,
    height="400px",
    center=[42.63991930229187, 0.029533360302191964],
    zoom=12,
    ee_objects=[image],
    vis_params=vis_params,
    labels=labels,
    label_position="topright",
)

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

![](https://i.imgur.com/S4qDRuA.jpeg)

# Question 3

Timeseries Inspector: Create a map with timeseries inspector to visualize USDA NASS Cropland Data Layers from 2010 to 2022. Add your name and USDA logo to the map



In [ ]:
from geemap.legends import builtin_legends

for legend in builtin_legends:
    print(legend)

NLCD
ESA_WorldCover
ESRI_LandCover
ESRI_LandCover_TS
Dynamic_World
NWI
MODIS/051/MCD12Q1
MODIS/006/MCD12Q1
GLOBCOVER
JAXA/PALSAR
Oxford
AAFC/ACI
COPERNICUS/CORINE/V20/100m
COPERNICUS/Landcover/100m/Proba-V/Global
USDA/NASS/CDL
ALOS_landforms


In [ ]:
Map3 = geemap.Map(center=[40, -100], zoom=4)

# Load dataset
collection = ee.ImageCollection('USDA/NASS/CDL').select('cropland').filterDate('2010', '2023')
vis_params = {'bands': ['cropland']}
years = collection.aggregate_array('system:index').getInfo()

# Add timeseries inspector
Map3.ts_inspector(
    left_ts=collection,
    right_ts=collection,
    left_names=years,
    right_names=years,
    left_vis=vis_params,
    right_vis=vis_params,
    width='80px',
)

# Add text to the map
text = "Created by Pablo Miranda"
Map3.add_text(text, fontsize=20, position='bottomright')

# Add a logo to the map
html = """
<img src="https://i.imgur.com/tzH2dNr.png">
"""
Map3.add_html(html, position="bottomleft")

# Add built-in legend
Map3.add_legend(
    title="USDA NASS Cropland Data (2010-2022)", builtin_legend='USDA/NASS/CDL', height='465px')

Map3

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Dropdown(layout=…

![](https://i.imgur.com/m53VFY8.jpeg)

# Question 4

Time slider: Create a map with the time slider to visualize Sentinel-2 for Knoxville, TN.


In [ ]:
# Load borders and filter by name
borders = ee.FeatureCollection("TIGER/2018/Counties")
knox = borders.filter(ee.Filter.eq('NAME', 'Knox'))
knox_tn = knox.filter(ee.Filter.eq('STATEFP', '47'))

# Get centroid
centroid = knox_tn.geometry().centroid().coordinates()
lonlat = centroid.getInfo()
latlon = lonlat[::-1]

Map4 = geemap.Map(center=latlon, zoom=12)

collection = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filter(ee.Filter.date('2020-06-01', '2022-07-01'))
    .filterBounds(ee.Geometry.Point(lonlat))
    .filterMetadata("CLOUDY_PIXEL_PERCENTAGE", "less_than", 11)
)

# Configura parametros de visualizacion
vis_params = {"min": 0, "max": 4000, "bands": ["B8", "B4", "B3"]}

# Add text to the map
text = "Created by Pablo Miranda"
Map4.add_text(text, fontsize=20, position='bottomright')

Map4.add_time_slider(collection, vis_params, time_interval=2)
Map4


Map(center=[35.993196068178555, -83.93723059189078], controls=(WidgetControl(options=['position', 'transparent…

![](https://i.imgur.com/oyuo2aJ.jpeg)


# Question 5
Split-panel Map: Use the following datasets to create a split-panel map for visualizing the ESA land cover data in the US. Add the ESA land cover legend to the map (Hint: the built-in legend for ESA land cover is ESA_WorldCover).

US Census States: ee.FeatureCollection("TIGER/2018/States")
ESA WorldCover 10m: ee.ImageCollection("ESA/WorldCover/v100")
Landsat: LANDSAT/LE7_TOA_5YEAR/1999_2003
Currently, the split-map control of ipyleaflet plotting backend has a bug (source). Use the folium plotting backend instead.



In [ ]:
import geemap.foliumap as geemap

Map5 = geemap.Map(center=[40, -100], zoom=4)

# Load bounds
states = ee.FeatureCollection("TIGER/2018/States")

# Load datasets
esa_lc = ee.ImageCollection('ESA/WorldCover/v100').first().clip(states)
landsat7 = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003').clip(states)

# Vis params
vis = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 120, 'gamma': 1.3}

# Create tiles
left_layer = geemap.ee_tile_layer(landsat7, vis, 'Landsat 7')
right_layer = geemap.ee_tile_layer(esa_lc, {}, 'ESA Land Cover')

# Create split map
Map5.split_map(left_layer, right_layer, add_close_button=True)

# Add text to the map
text = "Created by Pablo Miranda"
Map5.add_text(text, fontsize=20, position='bottomleft')

# Add built-in legend
Map5.add_legend(title='Land Cover Type', builtin_legend='ESA_WorldCover')

Map5

![](https://i.imgur.com/7BYBHKL.jpeg)